<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#새로-띄워진-창에서-로그인을-한다" data-toc-modified-id="새로-띄워진-창에서-로그인을-한다-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>새로 띄워진 창에서 로그인을 한다</a></span><ul class="toc-item"><li><span><a href="#학생-목록을-먼저-담아온다" data-toc-modified-id="학생-목록을-먼저-담아온다-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>학생 목록을 먼저 담아온다</a></span></li><li><span><a href="#댓글-긁어오기-함수" data-toc-modified-id="댓글-긁어오기-함수-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>댓글 긁어오기 함수</a></span></li><li><span><a href="#복습테스트-함수" data-toc-modified-id="복습테스트-함수-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>복습테스트 함수</a></span></li></ul></li><li><span><a href="#excel--불러와서-한-시트에-합치기" data-toc-modified-id="excel--불러와서-한-시트에-합치기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>excel  불러와서 한 시트에 합치기</a></span></li><li><span><a href="#특강반-가져오기" data-toc-modified-id="특강반-가져오기-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>특강반 가져오기</a></span><ul class="toc-item"><li><span><a href="#pds-함수-(옛날거)" data-toc-modified-id="pds-함수-(옛날거)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>pds 함수 (옛날거)</a></span></li></ul></li></ul></div>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import time
import requests
import re
import glob
import os
import re


driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)
driver.get('https://nid.naver.com/oauth2.0/authorize?svctype=0&response_type=code&client_id=C9hwybENgOtF&state=IWK3A3NUJQVTFVU2KJPK2DRAZ6L7OXGBFYZHHTH3GMTTXKI6A6ALVCF6SFNE3OLHQ4JVWLE6HFC7Q===&redirect_url=https%3A%2F%2Fauth.band.us%2Fexternal_account_login%3Ftype%3Dnaver')


# 새로 띄워진 창에서 로그인을 한다

## 학생 목록을 먼저 담아온다

In [31]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/85178180'
class_dic['PB3A'] = 'https://band.us/band/85178672'
class_dic['PD3A'] = 'https://band.us/band/85186142'
class_dic['PJ3A'] = 'https://band.us/band/85186150'
class_dic['PURPLE'] = 'https://band.us/band/85186110'
class_dic['RED'] = 'https://band.us/band/85186132'
class_dic['PDEE'] = 'https://band.us/band/85237203'
class_dic['WSCN'] = 'https://band.us/band/85232892'
class_dic['PSCN'] = 'https://band.us/band/85275199'

## class_dic['PB7A'] = 'https://band.us/band/84600978'
## class_dic['PD7A'] = 'https://band.us/band/84601004'

In [48]:
# 학생이름을 먼저 긁어놓는다... 

week = 1

member_count = []
member_dict = {}

for class_id in list(class_dic.keys()):
    
    url = class_dic[class_id]+'/member'
    driver.get(url)
    time.sleep(1.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    num = int(soup.select('div.uHeaderWrap > header > h1 > em')[0].text)-2
    member_count.append([class_id, num])
    
    member = soup.select('.ellipsis')[2:] # 위에 두줄은 admin, 이철 
    member_list = {}
    
    for i, name in enumerate(member): # 번호를 같이 뽑아줘야 딕셔너리에 들어감... 리스트로 넣을 경우 이름 수정이 힘들다. 
        who = name.text
        who = who.replace('\n', "")
        if class_id == 'PURPLE':
            who = who[5:].strip()
     
        member_list[i] = who
        
    member_list = pd.Series(member_list).str.extract(r'([가-힣].*)') # -> 시리즈에 str을 적용하면 데이터프레임이 된다. 
    member_dict[class_id] = member_list[0] # -> 다시 시리즈를 넣어줌... 

member11 = pd.DataFrame(member_dict)
member12 = pd.DataFrame(member_count, columns=['반명', '학생수'])

with pd.ExcelWriter(f'{week}주차_member.xlsx') as writer:  
    member11.to_excel(writer, sheet_name=f'{week}주차_학생명단', index=False)
    member12.to_excel(writer, sheet_name=f'{week}주차_학생수', index=False)
    
# 

In [49]:
# 정규표현식을 통해 미리 앞에서 처리 
# 이게 더 깔끔 !!!!

week = 2
p = re.compile('([가-힣].*)')

member_count = []
member_dict = {}

for class_id in list(class_dic.keys()):
    
    url = class_dic[class_id]+'/member'
    driver.get(url)
    time.sleep(1.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    num = int(soup.select('div.uHeaderWrap > header > h1 > em')[0].text)-2
    member_count.append([class_id, num])
    
    member = soup.select('.ellipsis')[2:] # 위에 두줄은 admin, 이철 
    member_list = [] # 여기 바꿈 
    
    for name in member: 
        who = name.text
        who = who.replace('\n', "")
        if class_id == 'PURPLE':
            who = who[5:].strip()
        
        who = p.findall(who)[0] #  리스트형태로 반환하기 때문에 첫번째 요소만 뽑아내기 (str) # 한글요소는 인덱싱 전에 해주기 ! 
 
        member_list.append(who)
    
    member_dict[class_id] = pd.Series(sorted(member_list))
    # 시리즈로 만들어서 넣어주면 길이가 안맞아도 로테이션 하지 않아도 된다. 
    # 그냥 리스트를 넣어주는 경우 길이가 안맞으면 불가.. 
    # 시리즈로 만들어주기 전에 미리 솔팅을 해줘야 한다... 

member11 = pd.DataFrame(member_dict) # 학생명단 
member12 = pd.DataFrame(member_count, columns=['반명', '학생수']) # 학생수 

with pd.ExcelWriter(f'{week}주차_member.xlsx') as writer:  
    member11.to_excel(writer, sheet_name=f'{week}주차_학생명단', index=False)
    member12.to_excel(writer, sheet_name=f'{week}주차_학생수', index=False)
    
# 

In [46]:
member_list.append(who)
member_list

['이영서', '이영서', '이영서']

In [38]:
member_dict["PA3A"].to_frame() # 아래 코드에서 쓰인 것... 

,0
0,강혜린
1,구준회
2,권영진
3,권지훈
4,김성준
5,김영현28
6,김채원
7,나소현
8,노건우
9,노재선


In [47]:
member11 = pd.DataFrame(member_dict)
member11.PA3A.to_frame()

,PA3A
0,강혜린
1,구준회
2,권영진
3,권지훈
4,김성준
5,김영현28
6,김채원
7,나소현
8,노건우
9,노재선


In [20]:
import re
who = 'WRED 송이정'
p = re.compile('([가-힣].*)')
a = p.findall(who)[0]
a

'송이정'

In [108]:
sorted(member_list)

['김도현',
 '김민정',
 '김예진',
 '김지희',
 '김현진',
 '김혜빈',
 '박수현',
 '서유빈',
 '서지원',
 '손여빈',
 '송예윤',
 '송이정',
 '오주연',
 '오지현',
 '유지연',
 '이규연',
 '이리사',
 '이솔지',
 '이승연',
 '이윤지',
 '이은서',
 '정수진',
 '정재연',
 '조민정',
 '최고은',
 '최수빈',
 '최지원',
 '한룻',
 '홍래영']

In [110]:
dictt ={}
for i, name in enumerate(member):
    dictt[i] = name

In [111]:
dictt

{0: <span class="ellipsis">RED(D)이리사</span>,
 1: <span class="ellipsis">WRED (D) 오지현</span>,
 2: <span class="ellipsis">WRED 김도현</span>,
 3: <span class="ellipsis">WRED 김민정</span>,
 4: <span class="ellipsis">WRED 김예진</span>,
 5: <span class="ellipsis">WRED 김지희</span>,
 6: <span class="ellipsis">WRED 박수현</span>,
 7: <span class="ellipsis">WRED 서유빈</span>,
 8: <span class="ellipsis">WRED 서지원</span>,
 9: <span class="ellipsis">WRED 손여빈</span>,
 10: <span class="ellipsis">WRED 송예윤</span>,
 11: <span class="ellipsis">WRED 송이정</span>,
 12: <span class="ellipsis">WRED 이규연</span>,
 13: <span class="ellipsis">WRED 이솔지</span>,
 14: <span class="ellipsis">WRED 이승연</span>,
 15: <span class="ellipsis">WRED 이윤지</span>,
 16: <span class="ellipsis">WRED 이은서</span>,
 17: <span class="ellipsis">WRED 정수진</span>,
 18: <span class="ellipsis">WRED 정재연</span>,
 19: <span class="ellipsis">WRED 조민정</span>,
 20: <span class="ellipsis">WRED 최고은</span>,
 21: <span class="ellipsis">WRED 최수빈</span>,
 22: <span clas

In [50]:
member = pd.read_excel('2주차_member.xlsx')

## 댓글 긁어오기 함수

In [149]:
def comment(m, w, p, v):
    
    
    pdss = [f'p-{m}-{w}-{i}' for i in range(1, p+1)]
    vocas = [f'v-{m}-{w}-{i}' for i in range(1, v+1)]
    test = f't-{m}-{w}' # 일주일에 한번 
    
    for class_id in list(class_dic.keys()):
        
        df = {}
        
        
        for pds in pdss:
            
            url = class_dic[class_id] + f'/hashtag/{pds}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(0.5)
        

                for _ in range(3):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')

                student_data = {}

                for student in students :

                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = student.select('button > strong')[0].text
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    else :
                        name = student.select('button > strong')[0].text

                    score = "O"

                    student_data[name] = score 

                df[pds] = student_data

    #             print(len(student_data))
            
            except:
                pass
        
        
        for voca in vocas:
            
            url = class_dic[class_id] + f'/hashtag/{voca}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(0.5)
        

                for _ in range(3):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')

                student_data = {}

                for student in students :

                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = student.select('button > strong')[0].text
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    else :
                        name = student.select('button > strong')[0].text
                    try: 
                        score = int(student.select('p')[0].text)
                    except :
                        score = ""
                        
                    student_data[name] = score 

                df[voca] = student_data
            
            except:
                pass
        
        test_df = get_test(class_id, test)
        
        member[class_id].name = 'name' # 프레임으로 변경시 컬럼 이름이 됨 
        df_name = member[class_id].to_frame() # 시리즈 형태로 저장된 밸류를 프레임으로 변환 
        df_name = df_name.dropna()
        
        
        total = pd.DataFrame(df).reset_index()
        total['index'] = total['index'].str.extract(r'([가-힣].*)') # 한글로 시작하는 시점부터 뒤까지 전부 불러와라 
        total = total.rename(columns={'index':"name"})
        
        total = pd.merge(df_name, total, on='name', how='outer')
        try : 
            total = pd.merge(total, test_df, on='name', how='outer')
        except:
            pass 
        
        total = total.fillna("X")
##         total['life'] = total.isin(["X"]).sum(axis=1)
##         total['life'] = total['life'].apply(lambda x : (7-x) * "🧡")
##         total = total.sort_values(by=['name'])
        total.to_excel(f'{class_id}_{week}주차.xlsx', index=False)
#     return total, df_name
#         with pd.ExcelWriter(f'{week}주차_comment11.xlsx', engine="openpyxl", mode='a') as writer:  
#             total.to_excel(writer, sheet_name=f'{class_id}', index=False)

In [118]:
class_dic = {}
# class_dic['PA3A'] = 'https://band.us/band/85178180'
# class_dic['PB3A'] = 'https://band.us/band/85178672'
# class_dic['PD3A'] = 'https://band.us/band/85186142'
# class_dic['PJ3A'] = 'https://band.us/band/85186150'
class_dic['PURPLE'] = 'https://band.us/band/85186110'
# class_dic['RED'] = 'https://band.us/band/85186132'
class_dic['PDEE'] = 'https://band.us/band/85237203'
# class_dic['WSCN'] = 'https://band.us/band/85232892'
# class_dic['PSCN'] = 'https://band.us/band/85275199'

## class_dic['PB7A'] = 'https://band.us/band/84600978'
## class_dic['PD7A'] = 'https://band.us/band/84601004'

In [120]:
# 돌리기 
week = 2
comment(9, 2, 0, 0) # 리턴값을 받아오려면 변수를 설정해야 함... 

In [52]:
def comment(m, w, p, v):
m = 9
w = 2
p = 4
pdss = [f'p-{m}-{w}-{i}' for i in range(1, p+1)]
pdss

['p-9-2-1', 'p-9-2-2', 'p-9-2-3', 'p-9-2-4']

In [70]:
# merge와 concat의 차이점... 
pd.concat([df_name, total], axis=1) # 인덱스를 기준으로 붙여버린다. merge 사용이 올바름 

,name,name,pds1회차,단어시험01회
0,권여진,송지원,O,49
1,김영현,김영현,O,49
2,김주희,남정우,O,46
3,김주희,김희윤,O,49
4,김지호,이장훈,O,48
5,김해일,김주희,O,
6,김희윤,설나영,O,45
7,남정우,권여진,O,48
8,마은서,박채린,O,44
9,민소예,민소예,O,50


In [124]:
# 아랫부분... 
        total = total.fillna("X")
##         total['life'] = total.isin(["X"]).sum(axis=1)
##         total['life'] = total['life'].apply(lambda x : (7-x) * "🧡")
##         total = total.sort_values(by=['name'])
        total.to_excel(f'{class_id}_{week}주차.xlsx', index=False)
#     return total, df_name

,PA3A,name,pds1회차,단어시험01회
0,강혜린,강혜린,O,50.0
1,구준회,구준회,O,NaN
2,권영진,권영진,O,49.0
3,권지훈,권지훈,NaN,50.0
4,김성준,김성준,O,NaN
5,김영현28,김영현28,O,NaN
6,김예림,김예림,O,47.0
7,김채원,김채원,O,46.0
8,나소현,나소현,O,50.0
9,노건우,노건우,O,45.0


## 복습테스트 함수

In [119]:
def get_test(class_id, tag):
    
    url = class_dic[class_id] + f'/hashtag/{tag}'
    driver.get(url)
    
    band_name = driver.find_element_by_class_name('uriText').text
    
    try : 
        # 퀴즈박스 클릭
        driver.find_element_by_class_name('item.-quiz').click() 


        scroll_area = driver.find_element_by_xpath('//*[@id="wrap"]/div[2]/div/div/section')
        action = webdriver.ActionChains(driver)
        action.move_to_element(scroll_area).perform()
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_area)
        time.sleep(1)


        # 퀴즈명 가져오기 
        quiz_name = driver.find_element_by_class_name('addTitle')
        quiz_name = quiz_name.text.replace("~", "_")



        driver.find_element_by_class_name('button._btnOpenTakerList').click() # 응답자 보기 클릭
        time.sleep(0.5)
        driver.find_element_by_class_name('btnMoreMenu._btnHeaderMoreMenu').click() # 점 3개 클릭
        time.sleep(0.5)
        driver.find_element_by_class_name('btnMenuText._btnDownloadGradingResult').click() # 채점결과 다운로드 

        time.sleep(2)


        # 저장된 파일 불러오기 
        add = f'C:/Users/hmkang/Downloads/{band_name}_채점결과_{quiz_name}_' + r'{}_{}.xlsx'
        filename = glob.glob(add.format("*", "*"))[0]
        rere = pd.read_excel(filename, header=3) # 3번째 줄부터 불러와라 

        # 멤버 및 점수만 뽑아내기 
        rere['멤버'] = rere['멤버'].str.extract(r'([가-힣].*)')
        rere = rere.dropna()
        rere = rere[['멤버', '점수']]
        rere = rere.rename(columns = {'멤버':'name', '점수':f'{tag}'})
        if class_id == "PURPLE":
            rere['name'] = rere['name'].apply(lambda x : x[6:])
        return rere 
    except :
        pass

In [106]:
a = '퍼플(B) 김나연'
rere3 = rere['name'].apply(lambda x : x[5:])

In [112]:
dff = pd.read_excel("PURPLE_2주차.xlsx")
dff['name'].apply(lambda x : x[5:])

0         
1         
2         
3         
4         
      ... 
81     정재윤
82     진수정
83     진휘현
84     최소현
85     황서영
Name: name, Length: 86, dtype: object

In [64]:
driver.find_element_by_class_name('btnMoreMenu._btnHeaderMoreMenu').click() # 점 3개 클릭
driver.find_element_by_class_name('btnMenuText._btnDownloadGradingResult').click() # 채점결과 다운로드 


In [34]:
pd.DataFrame.from_dict(member_dic, orient='index').T # 길이맞추라고 할때 해결볍 

,PA3A,PB3A,PD3A,PJ3A,PURPLE,RED
0,admin,admin,admin,admin,,admin
1,독해 이철,독해 이철,독해 이철,독해 이철,,독해 이철
2,PA3A 강혜린,PB3A 권여진,PD3A 김가현,PJ3A 김지훈,PLE(B) 이송하,RED(D)이리사
3,PA3A 구준회,PB3A 김영현,PD3A 김건영,PJ3A 최서영,강민지,WRED (D) 오지현
4,PA3A 권영진,PB3A 김주희,PD3A 김나연,PJ3A 김서연,국민희,WRED 김도현
5,PA3A 권지훈,PB3A 김주희,PD3A 김민선,PJ3A 김서윤,김다영,WRED 김민정
6,PA3A 김성준,PB3A 김지호,PD3A 김민지,PJ3A 김준석,김률,WRED 김예진
7,PA3A 김영현28,PB3A 김해일,PD3A 김성은,PJ3A 김준형,김리경,WRED 김지희
8,PA3A 김채원,PB3A 김희윤,PD3A 김소운,PJ3A 김지원,김미라,WRED 박수현
9,PA3A 나소현,PB3A 남정우,PD3A 박세훈,PJ3A 김지은,김민지39,WRED 서유빈


In [65]:
scroll_area = driver.find_element_by_xpath('//*[@id="wrap"]/div[2]/div/div/section')
action = webdriver.ActionChains(driver)
action.move_to_element(scroll_area).perform()
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_area)
time.sleep(1)

# excel  불러와서 한 시트에 합치기

각각 저장된 파일을 불러와서 merge한 다음 새로운 ❤ 부여

첫주차 버젼 

In [121]:
class_dic = {}
# class_dic['PA3A'] = 'https://band.us/band/85178180'
# class_dic['PB3A'] = 'https://band.us/band/85178672'
# class_dic['PD3A'] = 'https://band.us/band/85186142'
# class_dic['PJ3A'] = 'https://band.us/band/85186150'
# class_dic['PURPLE'] = 'https://band.us/band/85186110'
# class_dic['RED'] = 'https://band.us/band/85186132'
# class_dic['PDEE'] = 'https://band.us/band/85237203'
class_dic['WSCN'] = 'https://band.us/band/85232892'
class_dic['PSCN'] = 'https://band.us/band/85275199'

## class_dic['PB7A'] = 'https://band.us/band/84600978'
## class_dic['PD7A'] = 'https://band.us/band/84601004'

In [75]:
week = 1
week_dict = {}
week_merge = {}
for class_id in list(class_dic.keys()):
    week_dict[class_id] = pd.read_excel(f'{class_id}_{week}주차.xlsx')
    
for class_id in list(class_dic.keys()):
    df = week_dict[class_id]
    df['life'] = df.isin(["X"]).sum(axis=1)
    df['life'] = df['life'].apply(lambda x : (7-x) * "🧡")
    df = df.sort_values(by=['name'])
    week_merge[class_id] = df
    
with pd.ExcelWriter(f'{week}주차_comment.xlsx') as writer:  
    for class_id in week_dict.keys():
        week_dict[class_id].to_excel(writer, sheet_name=f'{class_id}', index=False)

그 다음 주차 부터.. 

In [122]:
week_pre_n = 1
week_now_n = 2


In [137]:
def full(x):
    if x == 15:
        return x+1
    else : 

In [148]:
full_score = 15
week_pre = {}
week_now = {}
week_merge = {}
for class_id in list(class_dic.keys()):
    week_pre[class_id] = pd.read_excel(f'./{week_pre_n}주차/{class_id}_{week_pre_n}주차.xlsx')
    week_now[class_id] = pd.read_excel(f'{class_id}_{week_now_n}주차.xlsx')

for class_id in list(class_dic.keys()):
    df = pd.merge(week_pre[class_id], week_now[class_id], on='name', how='outer')
    df['life'] = np.where(df['t-9-2'] == 15, df.isin(["X"]).sum(axis=1) -1, df.isin(["X"]).sum(axis=1))
            
    df['life'] = df['life'].apply(lambda x : (7-x) * "🧡")
    df = df.sort_values(by=['name'])
    week_merge[class_id] = df

with pd.ExcelWriter(f'{week_now_n}주차_comment.xlsx') as writer:  
    for class_id in week_merge.keys():
        week_merge[class_id].to_excel(writer, sheet_name=f'{class_id}', index=False)

In [147]:
import numpy as np
df['life'] = np.where(df['t-9-2'] == 15, df.isin(["X"]).sum(axis=1) -1, df.isin(["X"]).sum(axis=1))
df

,name,pds1회차,pds2회차,pds3회차,pds4회차,pds5회차,단어시험01회,단어시험02회,p-9-2-1,p-9-2-2,p-9-2-3,p-9-2-4,v-9-2-1,v-9-2-2,t-9-2,life
0,강혜린,O,O,O,O,O,50,50,O,O,O,O,50,50,14,0
1,구준회,O,O,O,O,O,X,X,O,O,O,O,X,X,X,5
2,권영진,O,O,O,O,O,49,49,O,O,O,O,49,48,13,0
3,권지훈,X,X,X,X,X,50,50,O,O,O,O,50,49,0,5
4,김성준,O,O,O,O,O,X,X,O,O,O,O,43,43,10,2
5,김영현28,O,O,O,O,O,X,X,O,O,O,X,45,46,X,4
6,김예림,O,O,O,O,O,47,48,O,O,O,O,46,42,12,0
7,김채원,O,O,O,O,O,46,45,O,O,O,O,49,49,12,0
8,나소현,O,O,O,O,X,50,48,O,O,O,O,50,48,13,1
9,노건우,O,O,O,O,O,45,46,O,O,O,O,49,49,14,0


In [135]:
df['three'] = (
    np.select(
        condlist=[df['one'] < 2, df['one'] < 4], 
        choicelist=[df['one'] * 10, df['one'] ** 2], 
        default=df['one'] + 10))
df

np.where(df["data"] <2.5, "False", "True")

2


In [ ]:
pd.merge(week_1['PA3A'], week_1['PB3A'], on='name', how='outer') 
# 데이터프레임 자체를 딕셔너리 형태로 저장해서 그것을 꺼내서 사용 가능하다.... 

In [137]:

df = pd.read_excel('1주차_comment.xlsx', sheet_name=None)


{'PA3A':      name pds1회차 pds2회차 pds3회차 pds4회차 pds5회차 단어시험01회 단어시험02회     life
 0     강혜린      O      O      O      O      O      50      50  🧡🧡🧡🧡🧡🧡🧡
 1     구준회      O      O      O      O      O       X       X    🧡🧡🧡🧡🧡
 2     권영진      O      O      O      O      O      49      49  🧡🧡🧡🧡🧡🧡🧡
 3     권지훈      X      X      X      X      X      50      50       🧡🧡
 4     김성준      O      O      O      O      O       X       X    🧡🧡🧡🧡🧡
 5   김영현28      O      O      O      O      O       X       X    🧡🧡🧡🧡🧡
 6     김예림      O      O      O      O      O      47      48  🧡🧡🧡🧡🧡🧡🧡
 7     김채원      O      O      O      O      O      46      45  🧡🧡🧡🧡🧡🧡🧡
 8     나소현      O      O      O      O      X      50      48   🧡🧡🧡🧡🧡🧡
 9     노건우      O      O      O      O      O      45      46  🧡🧡🧡🧡🧡🧡🧡
 10    노재선      O      O      O      O      O       X       X    🧡🧡🧡🧡🧡
 11    문하진      O      O      O      O      O      50      47  🧡🧡🧡🧡🧡🧡🧡
 12    박민아      O      O      O      O      O      46      44  🧡🧡🧡🧡🧡🧡

In [89]:
member_dict['PA3A'].name = 'name'
df_name = member_dict['PA3A'].to_frame()
df_name = df_name.iloc[2:,]
df_name.sort_values(by=['name'])

,name
2,강혜린
3,구준회
4,권영진
5,권지훈
6,김성준
7,김영현28
45,김예림
8,김채원
9,나소현
10,노건우


# 특강반 가져오기

In [167]:
class_dic = {}
class_dic['WSCN'] = 'https://band.us/band/85232892'
class_dic['PSCN'] = 'https://band.us/band/85275199'

m = 9
w = 2
s = 2
week = 2

In [158]:
spes = [f's-{m}-{w}-{i}' for i in range(1, s+1)]
spes

['s-9-2-1', 's-9-2-2']

In [169]:
for class_id in list(class_dic.keys()):
        
    df = {}

    spes = [f's-{m}-{w}-{i}' for i in range(1, s+1)]
    
    for spe in spes:

        url = class_dic[class_id] + f'/hashtag/{spe}'
        driver.get(url)
        time.sleep(1)

        try: 
            driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
            time.sleep(0.5)


            for _ in range(3):
                try:
                    driver.find_element_by_class_name('prevComment').click()
                    time.sleep(0.2)
                except :
                    pass

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            students = soup.select('.itemWrap')

            student_data = {}

            for student in students :

                
                name = student.select('button > strong')[0].text

                score = "O"

                student_data[name] = score 

            df[spe] = student_data
        
    
        except:
            pass
    df = pd.DataFrame(df).reset_index()   
    df.to_excel(f'{class_id}_{week}주차.xlsx', index=False)

In [166]:
df = pd.DataFrame(df)
df

,s-9-2-1,s-9-2-2
이규희,O,O
PA3A 이예은,O,O
PB3A 안효수,O,O
PA3A 윤준혁,O,O
PA3A 김성준,O,O
PA3A 채민,O,O
PA3A 강혜린,O,O
PA3A 김채원,O,O
PB3A 민소예,O,O
PD3A 배강식,O,O


In [165]:
df

{'s-9-2-1': {'이규희': 'O',
  'PA3A 이예은': 'O',
  'PB3A 안효수': 'O',
  'PA3A 윤준혁': 'O',
  'PA3A 김성준': 'O',
  'PA3A 채민': 'O',
  'PA3A 강혜린': 'O',
  'PA3A 김채원': 'O',
  'PB3A 민소예': 'O',
  'PD3A 배강식': 'O',
  'PA3A 최진웅': 'O',
  '윤건희': 'O',
  'PA3A 나소현': 'O',
  '이영서': 'O',
  'PD3A 이재영': 'O',
  'PJ3A 이윤지': 'O',
  'PA3A 안경수': 'O'},
 's-9-2-2': {'이규희': 'O',
  'PA3A 이예은': 'O',
  'PB3A 안효수': 'O',
  'PA3A 윤준혁': 'O',
  'PA3A 김성준': 'O',
  'PA3A 채민': 'O',
  'PA3A 강혜린': 'O',
  'PA3A 김채원': 'O',
  'PB3A 민소예': 'O',
  'PD3A 배강식': 'O',
  'PA3A 최진웅': 'O',
  '윤건희': 'O',
  'PA3A 나소현': 'O',
  '이영서': 'O',
  'PD3A 이재영': 'O',
  'PJ3A 이윤지': 'O',
  'PA3A 안경수': 'O'}}

## pds 함수 (옛날거)

In [ ]:
def pds(nums):
    
    field = 'PDS복습'
    
    for class_id in list(class_dic.keys()):
        
        student_data = []
        
        for num in nums:
            
            tagnum = f'{num}차'

            tag = f'{field}{tagnum}'

            url = class_dic[class_id] + f'/hashtag/{tag}'
            driver.get(url)
            time.sleep(1)

            try: 
                driver.find_element_by_class_name('commentToggle._commentToggleBtn').click()
                time.sleep(2)

                for _ in range(4):
                    try:
                        driver.find_element_by_class_name('prevComment').click()
                        time.sleep(0.2)
                    except :
                        pass



                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                students = soup.select('.itemWrap')


                for student in students :

                    name = student.select('button > strong')[0].text
                    if class_id == 'PURPLE':    # 퍼플은 반명을 한글로 적는 바람에 따로 해줘야 함 
                        name = name[5:].strip() # 전부 다 이거로 하게 되면 띄어쓰기 없는 애들은 솔팅 안됨 
                    score = 'O'

                    student_data.append([class_id, field, tagnum, name, score])

                print(len(student_data))
            
            except:
                pass

        columns = ['반명', '숙제이름', '회차', '이름', '점수']
        student_list = pd.DataFrame(student_data, columns=columns)
        student_list = student_list.drop_duplicates()
        student_list['이름'] = student_list['이름'].str.extract(r'([가-힣].*)') # 한글로 시작하는 시점부터 뒤까지 전부 불러와라 
        student_list.to_excel(f'{class_id}_{field}.xlsx', index=False)

In [ ]:
nums = list(range(1, 14))
pds(nums)

In [ ]:
v = 2
voca = [f'단어시험0{i}회' for i in range(1, v+1)]
voca

In [ ]:
students

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
students = soup.select('.itemWrap')


In [ ]:
data = []
for student in students :
    name = student.select('button > strong')[0].text
    if class_id == 'PURPLE':
        name = name[5:].strip()
    try :
        score = student.select('div.commentBody._commentModification > p')[0].text
    except :
        score =""
    data.append([name, score])
data

In [ ]:
students

In [ ]:
name = student.select('button > strong')[0].text
name

In [ ]:
score = student.select('p')[0].text

In [ ]:
students

In [ ]:
a = driver.find_element_by_css_selector('div.itemWrap > div.writeInfo > button > strong')

In [ ]:
student

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
pds = soup.select('div.itemWrap > div.writeInfo > button')

In [ ]:
name_list = []
# pds_n = input('태그를 입력하세요 :')
pds_n = 'PDS복습34차'

for name in pds: 
    who = name.text
    who = who.replace('\n', "")
    where, who = who.split(" ")
    what = pds_n

    name_list.append([what, where, who])

    
df = pd.DataFrame(name_list, columns=['숙제이름', '반명', '이름'])
df

In [ ]:
class_dic

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')




In [ ]:
# class_number = input("반명은? : ")
url = class_dic[class_number]+'/member'
# driver.get(url)
a = list(class_dic.keys())
a

In [ ]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/84576914'
class_dic['PB3A'] = 'https://band.us/band/84576918'
class_dic['PD3A'] = 'https://band.us/band/84576926'
class_dic['PJ3A'] = 'https://band.us/band/84576928'
class_dic['PURPLE'] = 'https://band.us/band/84570800'
class_dic['RED'] = 'https://band.us/band/84576846'
class_dic['PB7A'] = 'https://band.us/band/84600978'
class_dic['PD7A'] = 'https://band.us/band/84601004'


# class_number = input("반명은? : ")
member_list = []
for class_number in list(class_dic.keys()):
    url = class_dic[class_number]+'/member'
    driver.get(url)
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    member = soup.select('span.text.-flex')

    for name in member[1:]:
        who = name.text
        who = who.replace('\n', "")
        if class_number == 'PURPLE':
            who = who[5:].strip()
        where = class_number
        member_list.append([where, who])

member11 = pd.DataFrame(member_list, columns=['반명', '이름'])
member11['이름']=member11['이름'].str.extract(r'([가-힣].*)')
member11.to_excel('student_08.xlsx', index=False)

In [ ]:
member11['이름']=member11['이름'].str.extract(r'([가-힣].*)')

member11

In [ ]:
member11.to_excel('student_08.xlsx', index=False)

In [ ]:
class_dic = {}
class_dic['PA3A'] = 'https://band.us/band/84576914'
class_dic['PB3A'] = 'https://band.us/band/84576918'
class_dic['PD3A'] = 'https://band.us/band/84576926'
class_dic['PJ3A'] = 'https://band.us/band/84576928'
class_dic['PURPLE'] = 'https://band.us/band/84570800'
class_dic['RED'] = 'https://band.us/band/84576846'
class_dic['PB7A'] = 'https://band.us/band/84600978'
class_dic['PD7A'] = 'https://band.us/band/84601004'


In [ ]:
driver.add

In [ ]:
s = requests.Session()

In [ ]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
}
s.headers.update(headers)

In [ ]:
for cookie in driver.get_cookies():
    c = {cookie['name'] : cookie['value']}
    s.cookies.update(c)

In [ ]:
s.cookies


In [ ]:
response = s.get('https://band.us/band/84576914/hashtag/PDS%EB%B3%B5%EC%8A%B532%EC%B0%A8')
soup = BeautifulSoup(response.text, 'lxml')
soup

In [ ]:
for cookie in driver.get_cookies():
    driver.add_cookie(c)

In [ ]:
cks = driver.get_cookies()
cks